In [10]:
import os
from langchain.document_loaders import DirectoryLoader
from langchain.llms import VertexAI
from langchain import LLMChain
from langchain.document_loaders import DirectoryLoader , PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.core.display import Markdown
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate


# os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# os.environ["WANDB_PROJECT"] = "langchain-tracing"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ghc-001-7755ed5582db.json'


In [29]:
prompt_template = """
In this task, you'll be analyzing the resume of a job candidate. Your objective is to extract key information from all the resume of these people

Here is candidate's resume:

***

{resume}

***

Your task is to carefully read through the resume and below given information about the candidate. 

For example, if you're examining the resume of a software engineer named John Doe, your output might look like this in JSON format:
The JSON should include the following fields:

    \"Name\": \"John Doe\",
    \"Summary\": \"Software Engineer with 10 years of experience in developing scalable web applications. Proficient in Java, Python, C++.\",
    \"Work Experience\": \"Google Search, Facebook Ads, Uber\",
    \"Years Of Experience\": \"10 years\",
    \"Skills\": [\"Java\", \"Python\", \"C++\"],
    \"Projects\": [\"Google Search\", \"Facebook Ads\", \"Uber\"],
    \"Education\": \"Stanford University\",
    \"Certifications\": [\"AWS Certified Solutions Architect - Associate\", \"Google Cloud Certified Professional Cloud Architect\"],
    \"Awards and Achievements\": [\"Google Founders\' Award\", \"ACM Grace Hopper Award\"]


When you encounter a category for which the resume provides no information, please use None as the value.

The quality of your work will be assessed based on:

The completeness and accuracy of the information you extract from the resume.
The relevance of the extracted information to the job posting.
The correctness of the JSON format in your output.
Now, here's your question:


When you've completed your analysis, please provide the extracted details in JSON format.
"""

In [30]:
loader1 = PyPDFLoader('media/resumes/Vienna-Modern-Resume-Template.pdf')
pages = loader1.load()

# Get the content of the resume
resume = pages[0].page_content


In [31]:
promp_analysis_table  = PromptTemplate(template=prompt_template, input_variables=["resume"])
# Set up the language model
llm = VertexAI(model_name="text-bison@001") # select the model
llm_chain = LLMChain(prompt=promp_analysis_table, llm=llm)
output = llm_chain.run(resume = resume)
len(output)

465

In [27]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1400, chunk_overlap=100)
document_section = splitter.split_documents(docs)

NameError: name 'docs' is not defined

In [33]:
Markdown(output)

{
  "Name": "Janine Nel",
  "Summary": "Deadline- focused Sales Engineer with six years experience in technical sales, contributing to the 30% growth of income for regional technology solutions provider over five states, producing and assisting long-term contracts with Fortune 500 clients, and growing Territory sixfold up to $350M.",
  "Work Experience": [
    {
      "Company": "Engen Oil",
      "Position": "Sales Engineer",
      "Dates": "May 2022 — May 202